
# XML für die Korpuslinguistik — Workshop (90 Minuten) an der ZuKoKo

**Dozent**: Phillip B. Ströbel (UZH)  
**Ziele**: In 90 Minuten weisst du, …  
- was XML ist und warum es sich in der Korpuslinguistik bewährt,  
- was Header/Body bedeutet (z. B. TEI),  
- wie Tags/Attribute/Hierarchien funktionieren,  
- was der Unterschied zwischen flachem und tiefem XML ist,  
- welche Python-Bibliotheken sich für XML eignen und wie man einfache Analysen macht.

**Material**: Dieses Notebook findest du unter https://pstroe.github.io/xml-workshop-zukoko-2025/lab/index.html. Die Daten im Ordner `data/`.

Dieses Notebook verwendet **`SAC-Jahrbuch_1899_mul.tagged.xml`** als durchgehendes Beispiel.
Wir demonstrieren:
- Parsen & Traversieren (Elemente, Attribute, Hierarchie)
- Abfragen (XPath-ähnliche Pfade, optional `lxml`)
- Token-/POS-/Lemma-Analysen
- Type-Token-Ratio pro Artikel
- Konkordanzen & Kontext (KWIC)
- Export als CSV
- (Optional) einfache Visualisierung

> Hinweis: Plots nutzen `matplotlib` (keine Styles/Farben gesetzt).


## 0) Setup & Datei

In [ ]:

from pathlib import Path
import xml.etree.ElementTree as ET

SAC_PATH = Path(r'/data/SAC-Jahrbuch_1899_mul.tagged.xml")
print("Datei:", SAC_PATH.name, "Größe:", SAC_PATH.stat().st_size, "Bytes")

# Optional: lxml für komfortablere XPath
try:
    import lxml.etree as LET
    USING_LXML = True
except Exception:
    USING_LXML = False

USING_LXML



## 1) Parsen & erste Inspektion
Wir lesen das XML als Baum und schauen uns die oberste Struktur an.


In [ ]:

tree = ET.parse(SAC_PATH)
root = tree.getroot()
print("Root-Tag:", root.tag)
children = list(root)
print("Erste Kinder:", [c.tag for c in children[:5]])
articles = root.findall(".//article")
print("Anzahl Artikel:", len(articles))
if articles:
    first_article = articles[0]
    first_sents = first_article.findall(".//s")[:3]
    print("Beispiel: #Sätze im 1. Artikel (erste 3 gezeigt):", len(first_sents))
    if first_sents:
        print("Attr. des ersten Satzes:", first_sents[0].attrib)
        print("Erste 10 Wort-Tags des ersten Satzes:", [w.tag for w in first_sents[0].findall('.//w')[:10]])



## 2) Elemente, Attribute, Elementinhalt
Wir extrahieren bei `<w>`:
- **Elementinhalt** (Text)
- **Attribute**: `lemma`, `pos`

10 Beispiele:


In [ ]:

examples = []
for w in root.findall(".//w"):
    examples.append({
        "form (Elementinhalt)": (w.text or ""),
        "lemma": w.attrib.get("lemma"),
        "pos": w.attrib.get("pos")
    })
    if len(examples) >= 10:
        break
examples



## 3) Artikel-IDs, Sätze & Wörter zählen


In [ ]:

def iter_articles(root):
    for art in root.findall(".//article"):
        yield art

def article_id(art):
    return art.attrib.get("n")

def count_sent_words(art):
    sents = art.findall(".//s")
    words = art.findall(".//w")
    return len(sents), len(words)

summary = []
for art in iter_articles(root):
    sid, wid = count_sent_words(art)
    summary.append({"article_n": article_id(art), "sentences": sid, "words": wid})

import pandas as pd
df_summary = pd.DataFrame(summary).sort_values("article_n", key=lambda s: s.astype(str))
df_summary



## 4) Token-/Lemma-/POS-Tabellen


In [ ]:

rows = []
for art in iter_articles(root):
    a_id = article_id(art)
    for s in art.findall(".//s"):
        s_n = s.attrib.get("n")
        for w in s.findall(".//w"):
            rows.append({
                "article_n": a_id,
                "sent_n": s_n,
                "form": w.text or "",
                "lemma": w.attrib.get("lemma"),
                "pos": w.attrib.get("pos")
            })

import pandas as pd
df_tokens = pd.DataFrame(rows)
df_tokens.head(10)



## 5) Häufigkeiten (Formen, Lemmata, POS)


In [ ]:

form_freq = df_tokens["form"].value_counts().reset_index()
form_freq.columns = ["form", "freq"]
lemma_freq = df_tokens["lemma"].value_counts().reset_index()
lemma_freq.columns = ["lemma", "freq"]
pos_freq = df_tokens["pos"].value_counts().reset_index()
pos_freq.columns = ["pos", "freq"]

form_freq.head(20), lemma_freq.head(20), pos_freq.head(20)



## 6) Type-Token-Ratio (TTR) pro Artikel


In [ ]:

def ttr(series):
    tokens = series.dropna().tolist()
    if not tokens:
        return 0.0
    types = set(tokens)
    return len(types) / len(tokens)

ttr_by_article = (
    df_tokens.groupby("article_n")["form"]
    .apply(ttr)
    .reset_index(name="TTR_form")
    .merge(
        df_tokens.groupby("article_n")["lemma"].apply(ttr).reset_index(name="TTR_lemma"),
        on="article_n", how="left"
    )
)

ttr_by_article



## 7) Konkordanz / KWIC (Key Word in Context)
Einfacher KWIC für ein gegebenes **Lemma** oder eine **Form**.


In [ ]:

TARGET = "Jahrbuch"  # ändere auf ein Lemma oder eine Form
WINDOW = 5

def sent_tokens(s):
    return [w.text or "" for w in s.findall(".//w")]

def kwic_for_form(root, form, window=5, limit=20):
    out = []
    for s in root.findall(".//s"):
        toks = sent_tokens(s)
        for i, tok in enumerate(toks):
            if tok == form:
                left = " ".join(toks[max(0, i-window):i])
                right = " ".join(toks[i+1:i+1+window])
                out.append({"left": left, "kw": tok, "right": right, "sent_n": s.attrib.get("n")})
                if len(out) >= limit:
                    return out
    return out

kwic_for_form(root, TARGET, window=WINDOW, limit=20)



## 8) (Optional) XPath mit lxml
Wenn `lxml` verfügbar ist, zeigen wir ein XPath-Beispiel (alle Wörter mit `pos="NN"` im ersten Artikel).


In [ ]:

if USING_LXML:
    ltree = LET.parse(str(SAC_PATH))
    nodes = ltree.xpath("//article[1]//w[@pos='NN']")
    [n.text for n in nodes[:20]]
else:
    print("lxml nicht verfügbar — Abschnitt übersprungen.")



## 9) Visualisierung (einfach): POS-Verteilung insgesamt


In [ ]:

import matplotlib.pyplot as plt

pos_counts = df_tokens["pos"].value_counts().reset_index()
pos_counts.columns = ["pos", "freq"]
plt.figure()
plt.bar(pos_counts["pos"], pos_counts["freq"])  # keine Farben explizit setzen
plt.title("POS-Verteilung (gesamt)")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()



## 10) Export: Tokens als CSV


In [ ]:

out_csv = SAC_PATH.with_name("SAC_tokens.csv")
df_tokens.to_csv(out_csv, index=False, encoding="utf-8")
out_csv


## 11) Validierung gegen Schema

In [ ]:
# Pfad zur Schema-Datei (im gleichen Ordner wie die XML-Datei)
xsd_path = SAC_PATH.with_name("corpus.xsd")

# Lade XML und Schema
xml_doc = etree.parse(str(SAC_PATH))
with open(xsd_path, "rb") as f:
    xmlschema_doc = etree.parse(f)
xmlschema = etree.XMLSchema(xmlschema_doc)

# Validierung
is_valid = xmlschema.validate(xml_doc)
print("Validierungsergebnis:", is_valid)

# Falls Fehler auftreten, diese anzeigen
if not is_valid:
    for error in xmlschema.error_log:
        print(error.message, "→ Zeile", error.line)



## 11) Aufgaben (für die Teilnehmenden)
1. **POS je Artikel**: Erstelle eine Tabelle `article_n × pos` (Absolute / relative Häufigkeiten).
2. **Lemmata-Topliste**: Finde die häufigsten Lemmata pro Artikel (Top 10).
3. **Satzlängen**: Plot der Verteilung der Satzlängen (Anzahl Tokens pro Satz).
4. **Regex-Filter**: Finde alle Formen, die mit Großbuchstaben beginnen (Eigennamen-Heuristik).
5. **Export**: Speichere Ergebnisse als CSV.
